In [1]:
from pyspark.sql import SparkSession
from operator import add
import time

In [2]:
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.197:7077") \
        .appName("yijie_uc")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/16 23:01:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/16 23:01:24 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [3]:
data_frame = spark_session.read\
    .option("header", "true")\
    .json('hdfs://192.168.2.197:9000/user/hadoop/RC_2010-01')\
    .cache()

In [4]:
start_time = time.time()

In [5]:
data_frame.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [6]:
data_frame.count()

2884096

In [7]:
data_frame.rdd.getNumPartitions()

13

In [8]:
data_frame.show()

+--------+-------------+----------------------+-----------------+--------------------+----------------+-----------+-------------+-----+------+------+-------+--------+----------+----------+--------------+------------+-----+------------+-------------------+------------+---+
|archived|       author|author_flair_css_class|author_flair_text|                body|controversiality|created_utc|distinguished|downs|edited|gilded|     id| link_id|      name| parent_id|removal_reason|retrieved_on|score|score_hidden|          subreddit|subreddit_id|ups|
+--------+-------------+----------------------+-----------------+--------------------+----------------+-----------+-------------+-----+------+------+-------+--------+----------+----------+--------------+------------+-----+------------+-------------------+------------+---+
|    true|David_ungerer|                  null|             null|Good rant, stop l...|               0| 1262304000|         null|    0| false|     0|c0i12v3|t3_akcbt|t1_c0i12v3|  t3

In [9]:
df_subreddit = data_frame.select('subreddit')
df_subreddit = df_subreddit.dropna()
df_subreddit_frequency = df_subreddit.groupby("subreddit").count()
df_subreddit_f = df_subreddit_frequency.sort('count',ascending=False).show(20)

+-------------------+------+
|          subreddit| count|
+-------------------+------+
|          AskReddit|623825|
|         reddit.com|300476|
|               pics|198009|
|           politics|146922|
|               IAmA|135360|
|                WTF|114836|
|             gaming|113877|
|              funny| 80320|
|            atheism| 71549|
|            science| 70028|
|        programming| 66474|
|          worldnews| 65451|
|         technology| 49710|
|    DoesAnybodyElse| 37237|
|              Music| 26250|
|relationship_advice| 23163|
|fffffffuuuuuuuuuuuu| 20700|
|          Economics| 19635|
|             videos| 18558|
|              trees| 18314|
+-------------------+------+
only showing top 20 rows



In [10]:
print(f"Execution time: {time.time() - start_time}")

Execution time: 34.95212388038635
